In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

def convert_npy_to_image(npy_path, output_dir):
    # 加载 .npy 文件
    volume = np.load(npy_path)  # 形状 (1, 32, 256, 256)
    data = volume.squeeze(0)    # 变为 (32, 256, 256)

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 提取关键层面作为图像
    for idx in range(data.shape[0]):
        # 提取单层切片
        slice_data = data[idx]

        # 归一化到0-255范围
        normalized = (slice_data - np.min(slice_data)) / (np.max(slice_data) - np.min(slice_data)) * 255
        slice_image = normalized.astype(np.uint8)

        # 创建并保存图像
        img = Image.fromarray(slice_image)
        img.save(os.path.join(output_dir, f"slice_{idx:02d}.png"))

    # 选择有代表性的中层切片用于大模型分析
    mid_slice = data[data.shape[0] // 2]
    normalized = (mid_slice - np.min(mid_slice)) / (np.max(mid_slice) - np.min(mid_slice)) * 255
    Image.fromarray(normalized.astype(np.uint8)).save(os.path.join(output_dir, "mid_slice.png"))

    return os.path.join(output_dir, "mid_slice.png")

# 使用方法
ct_npy_path = "E:\\model\\img_data\\M3D_RefSeg_npy\\s0057\\ct.npy"
image_path = convert_npy_to_image(ct_npy_path, "E:\\model\\img_data\\M3D_RefSeg_npy\\s0057\\temp_images")

In [ ]:
import numpy as np
import matplotlib.animation as animation
from matplotlib import pyplot as plt

def create_volume_animation(npy_path, output_file):
    # 加载数据
    volume = np.load(npy_path).squeeze(0)

    # 创建动画
    fig, ax = plt.subplots()
    img = ax.imshow(volume[0], cmap='gray', animated=True)

    def update(i):
        img.set_array(volume[i])
        ax.set_title(f'Slice {i+1}/{volume.shape[0]}')
        return [img]

    ani = animation.FuncAnimation(fig, update, frames=volume.shape[0],
                                  interval=200, blit=True)

    # 保存为GIF
    ani.save(output_file, writer='pillow', fps=5)
    plt.close(fig)
    return output_file

# 使用方法
animation_path = create_volume_animation("E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\ct.npy", "E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\ct_animation.gif")

In [ ]:
import numpy as np
import cv2

# 1. 加载 .npy 文件
frames = np.load('E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\ct.npy')  # 形状应为 (T, H, W, C)

# 2. 设置视频参数
height, width = frames.shape[1:3]
fps = 30  # 帧率（根据需求调整）
output_file = 'E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\output_video.mp4'

# 3. 创建视频编码器
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 编码器
video_writer = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

# 4. 逐帧写入视频
for frame in frames:
    # 确保帧为 uint8 类型 [0, 255]
    if frame.dtype != np.uint8:
        frame = (frame * 255).astype(np.uint8)  # 若为浮点数 [0,1] 则转换

    # 转换色彩空间（若为RGBA则转RGB）
    if frame.shape[-1] == 4:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)

    video_writer.write(frame)

# 5. 释放资源
video_writer.release()
print(f"视频已保存至 {output_file}")

In [ ]:
#mp4
import numpy as np
import imageio

# 加载.npy文件
frames = np.load('E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\ct.npy')

print(f"原始数据形状: {frames.shape}")  # (1, 32, 256, 256)
print(f"数据类型: {frames.dtype}")      # float32
print(f"值范围: min={frames.min()}, max={frames.max()}")  # 0.0-1.0

# 1. 调整数据形状：移除第一个维度（批次维度）
frames = frames.squeeze(0)  # 现在形状为 (32, 256, 256)
print(f"调整后形状: {frames.shape}")

# 2. 转换为8位无符号整数 [0-255]
frames = (frames * 255).astype(np.uint8)

# 3. 添加通道维度（灰度图需要添加颜色通道）
# 创建RGB表示（所有通道相同）
frames_rgb = np.repeat(frames[..., np.newaxis], 3, axis=-1)
print(f"RGB形状: {frames_rgb.shape}")  # (32, 256, 256, 3)

# 4. 写入视频文件
imageio.mimwrite('E:\\model\\img_data\\M3D_RefSeg_npy\\s0000\\output_video.mp4',
                frames_rgb, fps=30, codec='libx264')

print("视频创建成功!")